In [14]:
import sqlite3
import csv
from datetime import datetime
def convertir_fecha(fecha):
    #Convertimos la fecha a date
    aux = datetime.strptime(fecha, '%d/%m/%Y %H:%M')
    #Convertimos la fecha al formato yyyy-mm-dd
    date = aux.strftime('%Y/%m/%d %H:%M')
    return date

def crear_bd(db_filename):
    conn = sqlite3.connect(db_filename)
    conn.execute("DROP TABLE IF EXISTS datos_generales")
    conn.execute("CREATE TABLE datos_generales (ticket TEXT PRIMARY KEY, nombre TEXT, indice TEXT, pais TEXT)")
    conn.execute("DROP TABLE IF EXISTS semanales_IBEX35")
    conn.execute("CREATE TABLE semanales_IBEX35 (ticket TEXT, fecha TEXT, precio REAL, FOREIGN KEY (ticket) REFERENCES datos_generales(ticket))")
    conn.commit()
    conn.close()

crear_bd("mercado.sqlite3")


def cargar_bd(db_filename, tab_datos, tab_ibex35):
    with open(tab_datos, 'r', newline='', encoding='utf8') as fich:
        datos_generales = csv.reader(fich, delimiter=';')
        lista_datos_generales = list(datos_generales)
    with open(tab_ibex35, 'r', newline='', encoding='utf8') as fich:
        datos_ibex = csv.reader(fich, delimiter=';')
        lista_datos_ibex = list(datos_ibex)
    conn = sqlite3.connect(db_filename)
    #Quitamos la cabecera
    tabla_generales=lista_datos_generales[1::]
    tabla_ibex = lista_datos_ibex[1::]
    for fila in tabla_ibex:
        fecha_original = fila[1]
        new_date = convertir_fecha(fecha_original)
        fila[1] = new_date
    for fila in tabla_generales:
    #Insertamos la fila en la tabla
       conn.execute("INSERT INTO datos_generales (ticket, nombre, indice, pais) VALUES (?, ?, ?, ?)", fila)
    for fila in tabla_ibex:
    #Insertamos la fila en la tabla
        conn.execute("INSERT INTO semanales_IBEX35 (ticket, fecha, precio) VALUES (?, ?, ?)", fila)
    conn.commit()
    conn.close()




cargar_bd("mercado.sqlite3", "Tabla1.csv", "Tabla2.csv")